# Generate job files for MSI computing
Main feature is to get all subfolders in a directory, and set up the output folders correspondingly

In [9]:
import myImageLib as mil
import os

In [69]:
# Construct args_list 
args_list = []
local_input = r'D:\Data\02182020\img'
remote_input = r'~\Data\Wei\03032020\10n0\8-bit'
remote_output = r'~\Data\02182020\img\result'
nd2Dirs = mil.dirrec(local_input, '*.nd2')
for nd2Dir in nd2Dirs:
    folder, file = os.path.split(nd2Dir)
    name, ext = os.path.splitext(file)
    input_dir = os.path.join(remote_input)
    output_dir = os.path.join(remote_output)
    args = ' '.join((input_dir, '')).replace('\\', '/') ### Pay attention here
    args_list.append(args)
# test master_input = I:\Github\Python\generic_proc\test_images
# test master_output = I:\Github\Python\generic_proc\test_images\result

In [70]:
code_path = r'~/code/Python/generic_proc/py_files'
py_name = r'to_tif.py'
walltime = int(len(args_list) * 0.1)# hr
nodes = 1
memory = 1 # gb

In [71]:
print('#!/bin/bash -l')
print('#PBS -l walltime={0}:00:00,nodes={1}:ppn=8,mem={2}gb'.format(walltime, nodes, memory))
print('#PBS -m abe')
print('#PBS -M liux3141@umn.edu\n')
print('cd {}'.format(code_path))
print('module load python3')
print('source activate pythonEnv')

for args in args_list:
    print('python {} {} &'.format(py_name, args))
print('wait')

#!/bin/bash -l
#PBS -l walltime=2:00:00,nodes=3:ppn=8,mem=1gb
#PBS -m abe
#PBS -M liux3141@umn.edu

cd ~/code/Python/generic_proc/py_files
module load python3
source activate pythonEnv
python to_tif.py ~/Data/Wei/02182020/img/No_1.nd2  &
python to_tif.py ~/Data/Wei/02182020/img/No_10.nd2  &
python to_tif.py ~/Data/Wei/02182020/img/No_10P.nd2  &
python to_tif.py ~/Data/Wei/02182020/img/No_1P.nd2  &
python to_tif.py ~/Data/Wei/02182020/img/No_2.nd2  &
python to_tif.py ~/Data/Wei/02182020/img/No_2P.nd2  &
python to_tif.py ~/Data/Wei/02182020/img/No_3.nd2  &
python to_tif.py ~/Data/Wei/02182020/img/No_3P.nd2  &
python to_tif.py ~/Data/Wei/02182020/img/No_4.nd2  &
python to_tif.py ~/Data/Wei/02182020/img/No_4P.nd2  &
python to_tif.py ~/Data/Wei/02182020/img/No_5.nd2  &
python to_tif.py ~/Data/Wei/02182020/img/No_5P.nd2  &
python to_tif.py ~/Data/Wei/02182020/img/No_6.nd2  &
python to_tif.py ~/Data/Wei/02182020/img/No_6P.nd2  &
python to_tif.py ~/Data/Wei/02182020/img/No_7.nd2  &
python to_t